<a href="https://colab.research.google.com/github/diwert-ai/First-step-in-NLP-HSE-bootcamp-/blob/main/kaggle%20competition/experiment_bias_postprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount google.drive and copy dataset to colab linux instance

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
INPUT_DIR = '/content/drive/MyDrive/kaggle/fsnlp_hse/input/'
OUTPUT_DIR = '/content/drive/MyDrive/kaggle/fsnlp_hse/output/'

WORK_DIR = '/content/'
EXPERIMENT = 'stats'
EXPERIMENT_DIR = OUTPUT_DIR + EXPERIMENT+ '/'

DATASET_DIR = INPUT_DIR
METADATA_DIR = INPUT_DIR

!mkdir {EXPERIMENT_DIR}

mkdir: cannot create directory ‘/content/drive/MyDrive/kaggle/fsnlp_hse/output/stats/’: File exists


# Install and set up Kaggle API

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!pip install -q kaggle --upgrade
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [ ]:
!kaggle competitions download -c hse-nlp-bootcamp
!unzip hse-nlp-bootcamp.zip

 82% 49.0M/59.8M [00:00<00:00, 110MB/s]
100% 59.8M/59.8M [00:00<00:00, 86.1MB/s]
Archive:  hse-nlp-bootcamp.zip
  inflating: new_test_ml.csv         
  inflating: sample_submission.csv   
  inflating: train_ml.csv            


# Imports

In [ ]:
import pandas as pd
import numpy as np
import random
import pprint as pp
import os


# import re
# from pymorphy2 import MorphAnalyzer
# from functools import lru_cache
# from nltk.corpus import stopwords

# from multiprocessing import Pool
# from tqdm import tqdm

import nltk
# nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, f1_score
from joblib import dump, load

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Configuration

In [ ]:
class Config:
  seed = 12536271
  samples_num = 51476
  experiment = EXPERIMENT
  test_size = 0.1

# Main pipeline starts here

In [ ]:
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)


set_seed(Config.seed)

## Load and process train data

In [ ]:
df = pd.read_csv('/content/train_ml.csv')
banks_list = list(df.bank.unique())
df['bank_id'] = df['bank'].apply(lambda x: banks_list.index(x))
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y %H:%M')
df = df.dropna()
df = df.astype({'grades': 'int32'})
df.head()

,bank,feeds,grades,date,bank_id
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1,2017-02-16 16:10:00,0
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2,2016-12-13 01:05:00,1
4,promsvyazbank,"Имею потребительский кредит, взятый в Связь-ба...",2,2020-08-04 06:38:00,4
7,homecreditbank,Хочу выразить глубокую благодарность всем сотр...,5,2015-04-19 20:11:00,7
8,vtb,"6 марта 2019г. В 10:33 при проходе в метро ""Ку...",2,2019-03-28 18:36:00,3


In [ ]:
df_len=len(df)
df_len

51476

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
def get_numeric_data(x):
    return x[['bank_id']]

def get_text_data(x):
    return x['text']

In [ ]:
model = load( OUTPUT_DIR+'logreg_5f/model_logreg_5f_[fold_1]_[sn_51476]_[sd_12536271]_[cv_0.7883].joblib')

In [ ]:
df['text'] = df.feeds.apply(lambda x: x.lower())

In [ ]:
proba = model.predict_proba(df[['bank_id', 'text']])

In [ ]:
preds = np.argmax(proba, axis=1) + 1

In [ ]:
gt = df['grades']

In [ ]:
gt

0        1
1        2
2        2
3        5
4        2
        ..
51471    1
51472    2
51473    1
51474    3
51475    1
Name: grades, Length: 51476, dtype: int32

In [ ]:
f1_score(gt, preds, average='micro')

0.8161279042660657

In [ ]:
errors_idx = np.where(preds!= gt)

In [ ]:
df['preds'] = preds

In [ ]:
df[['c1', 'c2', 'c3', 'c4', 'c5']] = proba

In [ ]:
df[['feeds','grades','preds','c1', 'c2', 'c3', 'c4', 'c5']]

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
df_errors = df[['feeds','grades','preds','c1', 'c2', 'c3', 'c4', 'c5']].iloc[errors_idx]

In [ ]:
str_query_12_54 = '((grades==1 | grades==2) & (preds==5 | preds==4)) | ((grades==5 | grades==4) & (preds==1 | preds==2))'
str_query_1_5 = '((grades==1) & (preds==5) | ((grades==5) & (preds==1)))'
str_query_1_2 = '((grades==1) & (preds==2) | ((grades==2) & (preds==1)))'

In [ ]:
df_errors.query(str_query_1_2)

,feeds,grades,preds,c1,c2,c3,c4,c5
1,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12.2016 г. с 13 часов до 15. В данном офисе не оказалось ни одного компетентного сотрудника по нашему вопросу. 1,5 часа ожидания - и итог - мы ничем не можем вам помочь, обратитесь в другое отделение. Не могли внести денежные средства по кредиту АО Рост Банк (который так понимаю входит теперь в состав Бинбанка), запутались в реквизитах, спорили сотрудники между собой о схемах внесения - не смогли прийти к общему знаменателю. Позвонили на горячую линию дабы оставить заявку на работу данного офиса, сначала ответил вежливый сотрудник, принёс свои извинения и переключил на сотрудника по обработке претензий, который пояснил, что если я не являюсь клиентам банка, то и возможности составить заявку у меня нет. Выходит если у вас нет вклада или кредита в данном банке, то клиентом вы не являетесь и оставить негативный отзыв или составить заявку не сможете. Отправила смс-отзыв на номер указанный на буклетах на информационной стойке - +7 (916) 00*-**-95.",2,1,0.715416,0.245029,0.012618,0.012566,0.014371
2,"Имею потребительский кредит, взятый в Связь-банке и перешедший к Промсвязьбанку. В мае 2020 года внес дополнительно сумму для частичного досрочного погашения с условием уменьшения срока платежа. Сумма пробная, всего 3000, чтобы понять алгоритм. В итоге: кредит с ежемесячным платежом 14000 до 31 января 2023 года. Последний платеж по старому графику 31.01.2023 в сумме 12188 руб, после внесения дополнительных средств - последний платеж в тот же месяц 31.01.2023 в сумме 21500. Т.е. увеличился в 2 раза. При этом платежа в декабре 2022 года нет. Обратился в ДО Октябрьское поле за поЯснением. Сотрудники объяснили, что т.к. внесённые 3000 рублей срок уменьшить не могут, то банк просто объединил 2 последних платежа. Почему пропущен декабрь 2022 года пояснить не смогли. Сразу поясню, досрочно нашу ипотеку в другом банке. Любая дополнительно временная сумма учитывается, если она меньше месячного платежа - уменьшается сумма последнего платежа.Самое главное: предложили написать запрос. Написал. 17.06.20 написал заявление. Заверено менеджером по работе с клиентом. Оригинал на руках. По прошествии месяца обратился опять, пояснить методику расчета при досрочном частичном погашении мне не смогли. На вопрос : ""Где ответ?"" Получил ответ, что я должен был по горячей линии отслеживать его. Сделали повторную копию. Наверное, отправили повторно.Вопрос:1. Правила работы с обращениями к банку согласно закона для ПСБ регламентированы какими документами? 2. Прошу пояснить, почему в отношении меня принят такой график платежей.3. Куда мне писать, чтобы получить внятное объяснение по графику платежей и расчета суммы?",2,1,0.634805,0.321222,0.031603,0.006510,0.005859
4,"6 марта 2019г. В 10:33 при проходе в метро ""Курская"" по социальной карте (бесплатный проезд), у меня с карты были сняты деньги за проезд в метро. примерно через час я обратился в офис банка,где написал жалобу (претензию) на необоснованное списание средств с карты и попросил вернуть деньги. Претензию у меня приняли и сказали,что до 28 марта мне сообщат о решении. Ответа я так и не дождался и 28 марта позвонил в банк сам. Ответ ошеломил и потряс меня. Оказалось,что я сам виноват в том,что у меня списали деньги за проезд.Нужно проходить льготникам в определенные турникеты. Хотя на станциях Московского метрополитена это не указано,в частности на станции метро ""Курская"" и я всегда проходил до этого без проблем. И откуда то у меня оказалась подключена услуга по бесконтактной оплате проезда. Деньги мне вернуть отказались. Сразу скажу претензий к сотрудникам офиса на Земляном Валу,27 у меня нет.В отношении меня они вели себя очень корректно.А вот сам банк ВТБ поступил в отношении меня бесчеловечно. Теперь боюсь пользоваться услугами этого банка по своей социальной карте,на следующий раз кто -нибудь другой спишет еще деньги с моей карты и снова клиент будет виноват.",2,1,0.527840,0.234863,0.027015,0.048294,0.161988
6,"04.03.

In [ ]:
probs_file = OUTPUT_DIR+'logreg_5f/'+'final_probs_logreg_5f_[sn_51476]_[sd_12536271]__[f1_0.7894].npy'
probs_file

'/content/drive/MyDrive/kaggle/fsnlp_hse/output/logreg_5f/final_probs_logreg_5f_[sn_51476]_[sd_12536271]__[f1_0.7894].npy'

In [ ]:
probs_file = OUTPUT_DIR+'logreg_5f/'+'probs_logreg_5f_[fold_1]_[sn_51476]_[sd_12536271]_[cv_0.7883].npy'
probs_file

'/content/drive/MyDrive/kaggle/fsnlp_hse/output/logreg_5f/probs_logreg_5f_[fold_1]_[sn_51476]_[sd_12536271]_[cv_0.7883].npy'

In [ ]:
probs = np.load(probs_file)

In [ ]:
probs

array([[0.56497903, 0.19388107, 0.0346407 , 0.01987401, 0.18662518],
       [0.75861711, 0.05285007, 0.04866351, 0.04507071, 0.0947986 ],
       [0.88960598, 0.07285473, 0.01743578, 0.00435706, 0.01574645],
       ...,
       [0.90970383, 0.06731794, 0.01185953, 0.00374519, 0.00737351],
       [0.79551931, 0.14388386, 0.04496204, 0.0053567 , 0.01027809],
       [0.74158601, 0.16832768, 0.03536649, 0.01297022, 0.0417496 ]])

In [ ]:
preds = np.argmax(probs, axis=1) + 1

In [ ]:
preds_len = len(preds)
preds_len

17220

In [ ]:
bias = (0.97, 1.05, 1.05, 1.05, 0.97)
probs_corr = np.multiply(probs, bias)
preds_corr = np.argmax(probs_corr, axis=1) + 1
for i in range(5):
  print(f'class {i+1}: {len(df[df["grades"] == i + 1])/df_len: 0.4f} {sum(preds_corr == i + 1)/preds_len: 0.4f}')

class 1:  0.5389  0.6911
class 2:  0.1094  0.0142
class 3:  0.0458  0.0018
class 4:  0.0295  0.0027
class 5:  0.2764  0.2902


In [ ]:
test = pd.read_csv('/content/new_test_ml.csv', index_col=0)
sub = pd.DataFrame({'inds': test.index,
                    'grades': preds_corr})
bias_tag = '[bias_'+'_'.join(f'{value:.2f}' for value in bias)+']'
bias_tag

'[bias_0.97_1.05_1.05_1.05_0.97]'

# Save results to files

In [ ]:
suffix = f'{Config.experiment}_[sn_{Config.samples_num}]_[sd_{Config.seed}]_'+'_1f_'+bias_tag
sub_filename = 'sub_'+suffix+'.csv'
probs_filename = 'probs_'+suffix+'.npy'
sub.to_csv(sub_filename, index=False)
np.save(probs_filename, probs)
!cp {sub_filename} {EXPERIMENT_DIR}
!cp {probs_filename} {EXPERIMENT_DIR}

# Submit predictions

In [ ]:
!kaggle competitions submit -c hse-nlp-bootcamp  -f {'/content/'+sub_filename} -m "Submission via API"

100% 124k/124k [00:00<00:00, 224kB/s]
Successfully submitted to HSE | NLP Bootcamp | Sentiment Analysis